In [1]:
import vizdoom as vzd
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
possible_actions = np.zeros((6,4),dtype=int).tolist()
possible_actions[0] = [1,0,0,0]
possible_actions[1] = [0,1,0,0]
possible_actions[2] = [0,0,-45,0]
possible_actions[3] = [0,0,45,0]
possible_actions[4] = [0,0,0,10]
possible_actions[5] = [0,0,0,-10]

possible_actions

[[1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, -45, 0],
 [0, 0, 45, 0],
 [0, 0, 0, 10],
 [0, 0, 0, -10]]

In [3]:
import numpy as np

def is_visible_segment(raw_map, start_row, end_row, col):
    """Проверяет видимость в сегменте по заданному столбцу."""
    for row in range(start_row, end_row + 1):
        if raw_map[row, col, 0] == 255:  # Если пиксель белый
            return False
    return True

def process_segment(raw_map, start_row, end_row, col_start, col_end, map):
    """Обрабатывает сегмент карты для определения видимости."""
    for col in range(col_end,col_start-1,-1):
        # Проверяем видимость сверху
        if is_visible_segment(raw_map, start_row, end_row, col):
            map[start_row, col, :] = [0, 0, 0]  # Обозначаем как видимый

def divide_and_conquer(raw_map: np.array):
    # Заменяем значения больше 100 на белый цвет
    raw_map[raw_map[:, :, 0] > 100] = 255
    raw_map[raw_map[:, :, 1] > 100] = 255
    raw_map[raw_map[:, :, 2] > 100] = 255
    raw_map[118:121, 158:161, :] = [255, 0, 0]

    map = np.ones_like(raw_map) * 255

    # Определяем границы для обработки
    height = raw_map.shape[0]
    width = raw_map.shape[1]

    # Разделяем и обрабатываем по строкам
    for row in range(120, -1, -1):
        left_col = -1
        right_col = -1
        
        # Ищем правую границу видимости
        for j in range(159, width):
            if raw_map[row, j, 0] != 255:
                right_col = j
                break
        
        # Ищем левую границу видимости
        for j in range(158, -1, -1):
            if raw_map[row, j, 0] != 255:
                left_col = j
                break
        
        if right_col != -1:
            map[row, right_col, :] = [0, 0, 0]
        if left_col != -1:
            map[row, left_col, :] = [0, 0, 0]

    # Обработка столбцов для видимости снизу
    for col in range(width):
        top_row = -1
        
        for i in range(118, -1, -1):
            if raw_map[i, col, 0] != 255:
                top_row = i
                break
        
        if top_row != -1:
            map[top_row, col, :] = [0, 0, 0]

    #map[118:121, 158:161,:] = [255,0,0]
    
    return map

# Пример использования функции
# raw_map должен быть загружен заранее как NumPy массив.
# processed_map = divide_and_conquer(raw_map)

In [4]:
def map_processing(raw_map: np.array):
    raw_map[raw_map[:,:,0] > 100] = 255  # Replace all values greater than 100 with 255
    raw_map[raw_map[:,:,1] > 100] = 255  # Replace all values greater than 100 with 255
    raw_map[raw_map[:,:,2] > 100] = 255  # Replace all values greater than 100 with 255
    raw_map[118:121,158:161,:] = [255,0,0]

    map = np.ones_like(raw_map)*255
    for i in range(120,-1,-1):
        j = 159
        while j!=320 and raw_map[i,j,0] == 255:
            j += 1
        if j!=320:
            map[i,j,:] = [0,0,0]
        j = 158
        while j!=0 and raw_map[i,j,0] == 255:
            j -= 1
        if j!=-1:
            map[i,j,:] = [0,0,0]
    for j in range(0,raw_map.shape[1]):
        i = 118
        while i!=0 and raw_map[i,j,0] == 255:
            i -= 1
        if i!=0:
            map[i,j,:] = [0,0,0]
    map[118:121,158:161,:] = [255,0,0]        
    return map
# duration 4 tics
def get_pictures(game: vzd.DoomGame):
    state = game.get_state()    
    st = state.screen_buffer.transpose(1,2,0)
    img = np.array([st])
    # turn left
    game.make_action(possible_actions[2])
    state = game.get_state()
    st = state.screen_buffer.transpose(1,2,0)
    img = np.append(img, [st], axis=0)
    dm = state.automap_buffer.transpose(1,2,0)
    debug_map = np.array([dm])
    # turn right
    game.make_action(possible_actions[3],2)
    state = game.get_state()
    st = state.screen_buffer.transpose(1,2,0)
    img = np.append(img, [st], axis=0)
    dm = state.automap_buffer.transpose(1,2,0)
    debug_map = np.append(debug_map, [dm], axis=0)
    # return to center
    game.make_action(possible_actions[2])
    state = game.get_state()
    dm = state.automap_buffer.transpose(1,2,0)
    debug_map = np.append(debug_map, [dm], axis=0)
    auto_map = state.automap_buffer.transpose(1,2,0)
    #processed_map = map_processing(auto_map)
    processed_map = divide_and_conquer(auto_map)
    #processed_map = tracing(auto_map)
    return img, auto_map, processed_map,debug_map

In [6]:
game = vzd.DoomGame()
game.load_config('resources/temp_maps/datagen.cfg')
game.set_automap_mode(vzd.AutomapMode.NORMAL)

for i in range(100):
    game.init()
    game.new_episode()
    state = game.get_state()
    st = state.screen_buffer.transpose(1,2,0)
    img = np.array([st])
    am = state.automap_buffer.transpose(1,2,0)
    depth = np.array([state.depth_buffer])
    game.set_automap_mode(vzd.AutomapMode.NORMAL)
    tmp_img, auto_map, tmp_processed_map, debug_map = get_pictures(game)
    tmp_position = state.game_variables
    if i==0:
        images = np.array([tmp_img])
        maps = np.array([tmp_processed_map])
        positions = np.array([tmp_position])
    else:
        images = np.append(images,[tmp_img],axis=0)
        maps = np.append(maps,[tmp_processed_map],axis=0)
        positions = np.append(positions,[tmp_position],axis=0)    
    game.close()

In [14]:
fig,ax = plt.subplots(25,4,figsize=(200,100))
for i in range(25):
    for j in range(4):
        ax[i,j].imshow(maps[i])

In [11]:
[print(f'x={a[0]}, y={a[1]}, z={a[2]}, angle={a[3]}') for a in positions]

x=1043.1000061035156, y=60.20799255371094, z=0.0, angle=13.886718753233254
x=460.3260040283203, y=-596.1199951171875, z=0.0, angle=42.550048838031955
x=575.3719940185547, y=-171.77000427246094, z=0.0, angle=325.44799812264927
x=1043.1000061035156, y=60.20799255371094, z=0.0, angle=13.886718753233254
x=460.3260040283203, y=-596.1199951171875, z=0.0, angle=272.09838873522784
x=1043.1000061035156, y=60.20799255371094, z=0.0, angle=13.886718753233254
x=694.1900024414062, y=58.32200622558594, z=0.0, angle=95.2679443581188
x=1043.1000061035156, y=60.20799255371094, z=0.0, angle=13.886718753233254
x=466.9830017089844, y=-466.7489929199219, z=0.0, angle=286.5783692073492
x=1043.1000061035156, y=60.20799255371094, z=0.0, angle=13.886718753233254
x=1043.1000061035156, y=60.20799255371094, z=0.0, angle=13.886718753233254
x=240.0, y=-176.0, z=0.0, angle=280.06347662770736
x=1043.1000061035156, y=60.20799255371094, z=0.0, angle=13.886718753233254
x=1043.1000061035156, y=60.20799255371094, z=0.0, an

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [15]:
np.savez('data/mywayhome_data.npz', images=images, maps=maps,positions=positions)

In [ ]:
data = np.load('data/mywayhome_data.npz')
d1 = data['maps']
d2 = data['images']
d3 = data['positions']